In [38]:
# import h5 reader
import h5py
import numpy as np
h5_file = '/Users/shengduoliu/Pylith_V2.0/pylith-2.2.2-darwin-10.13.6/src/pylith-2.2.2/examples/3d/Vito/usualSample/output2/h5Outputs/DRS1_1.5Load17_Vw1.5_fw0.1_theta0.08-fault.h5'
f = h5py.File(h5_file, 'r')

In [59]:
# Get time
time = np.array(f['time']).reshape([-1])
time = time - np.min(time)
nOfTSteps = time.shape[0]

# Get Slip rates
SlipRates = np.array(f['vertex_fields']['slip_rate'])
Vx = SlipRates[:, :, 0].transpose()
Vy = SlipRates[:, :, 1].transpose()
nOfNodes = Vx.shape[0]

# Find the Fourier coefficients
FourierTerms = 16
T = np.max(time)

# Compute the Fourier terms
Ks = np.array(range(FourierTerms))
coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

# Compute the fourier coefficients
print('time.shape: ', time.shape)
Observations = np.trapz(VxcoskPiTt, x=time)

time.shape:  (119,)


In [61]:
Observations.shape

(47616,)

In [45]:
for i in coskPiTt:
    print(i.shape[0])

119
119
119
119
119
119
119
119
119
119
119
119
119
119
119
119


In [19]:
v = np.array(f['vertex_fields']['slip_rate'])
v.shape

(119, 2976, 3)

In [20]:
v

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.82009980e-06, -2.51043170e-14,  8.67360871e-22],
        [ 3.82009840e-06,  3.25609626e-14, -1.08420307e-21],
        [ 3.82009607e-06,  2.10998685e-14, -1.51788275e-21],
        ...,
        [ 3.82008960e-06, -1.72044648e-14, -2.16840435e-22],
        [ 3.82009799e-06,  4.07680125e-14, -4.33682523e-22],
        [ 3.82010054e-06, -2.98239872e-14,  8.67361576e-22]],

       [[ 3.82004958e-06,  5.55840225e-14, -8.67359819e-22],
        [ 3.82005327e-06,  1.55064501e-14, -4.25254733e-28],
        [ 3.82005464e-06, -1.43306496e-14, -1.95809966e-28],
        ...,
        [ 3.82005984e-06, -1.25930115e-14,